In [1]:
import pickle
import os
import boto3
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, select
import os
import pickle
from nltk.corpus import stopwords
from nltk import RegexpTokenizer

In [2]:
engine_name = 'postgresql://' + str(os.environ['zU']) + ':' + str(os.environ['zP']) + '@13.59.54.149:5432/project_kojak'
engine = create_engine(engine_name)

In [3]:
df = pd.read_sql_query('''SELECT 
                           tracks.album_art,
                           tracks.album_name,
                           tracks.duration_ms,
                           tracks.energy,
                           tracks.liveness,
                           tracks.loudness,
                           tracks.speechiness,
                           tracks.tempo,
                           tracks.track_id,
                           tracks.track_name,
                           artists.artist_popularity,
                           artists.artist_name,
                           artists.artist_id,
                           lyrics.lyrics 
                           FROM tracks
                           JOIN artists ON tracks.artist_id = artists.artist_id
                           JOIN lyrics ON tracks.track_id = lyrics.track_id
                           WHERE tracks.energy IS NOT NULL
                           AND lyrics.lyrics IS NOT NULL
                           ORDER BY tracks.track_name;''', engine)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20359 entries, 0 to 20358
Data columns (total 14 columns):
album_art            20359 non-null object
album_name           20359 non-null object
duration_ms          20359 non-null int64
energy               20359 non-null float64
liveness             20359 non-null float64
loudness             20359 non-null float64
speechiness          20359 non-null float64
tempo                20359 non-null float64
track_id             20359 non-null object
track_name           20359 non-null object
artist_popularity    20359 non-null int64
artist_name          20359 non-null object
artist_id            20359 non-null object
lyrics               20359 non-null object
dtypes: float64(5), int64(2), object(7)
memory usage: 2.2+ MB


In [5]:
import string

In [6]:
df['track_name'] = df['track_name'].apply(lambda i: string.capwords(i))

In [7]:
df.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20073 entries, 0 to 20358
Data columns (total 14 columns):
album_art            20073 non-null object
album_name           20073 non-null object
duration_ms          20073 non-null int64
energy               20073 non-null float64
liveness             20073 non-null float64
loudness             20073 non-null float64
speechiness          20073 non-null float64
tempo                20073 non-null float64
track_id             20073 non-null object
track_name           20073 non-null object
artist_popularity    20073 non-null int64
artist_name          20073 non-null object
artist_id            20073 non-null object
lyrics               20073 non-null object
dtypes: float64(5), int64(2), object(7)
memory usage: 2.3+ MB


In [9]:
df.reset_index(drop=True, inplace=True)

In [10]:
df.drop(df[df['lyrics'].str.contains('<span')].index, inplace=True)

In [11]:
df.reset_index(drop=True, inplace=True)

In [12]:
len(df)

19917

In [13]:
with open('../Data_Files/all_lyrics_dataframe.pickle', 'wb') as f:
    pickle.dump(df, f)

In [14]:
s3 = boto3.client('s3', 
                  aws_access_key_id=os.environ['AWS_ACCESS_KEY'], 
                  aws_secret_access_key=os.environ['AWS_SECRET_KEY'])

In [15]:
s3.upload_file('../Data_Files/all_lyrics_dataframe.pickle', 'metis-project-kojak-bucket', 'all_lyrics_dataframe.pickle')